In [1]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import requests
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
# URL of page to be scraped; need to add each state to end of URL in loop
base_url = 'https://en.wikipedia.org/wiki/List_of_radio_stations_in_'

In [3]:
# CSV file of states for loop
states_df = pd.read_csv("List_of_states.csv")
states_df.head()

,States_w_space,State_w_no_space
0,Alabama,Alabama
1,Alaska,Alaska
2,Arizona,Arizona
3,Arkansas,Arkansas
4,California,California


In [4]:
# Initiate radio_df
url = base_url + states_df.State_w_no_space[0]
# read url
table = pd.read_html(url)
# Create a Dateframe From Table
radio_temp_df = pd.DataFrame(table[0])

#drop rows
radio_temp_df = radio_temp_df.drop(radio_temp_df.index[:])

# Loop through states
for index, row in states_df.iterrows():
    #Combine base url and state
    url = base_url + states_df.State_w_no_space[index]
    # read url
    table = pd.read_html(url)
    # Create a Dateframe From Table
    table_df = pd.DataFrame(table[0])
    # Add State
    table_df[5] = states_df.States_w_space[index]
    # append to radio_df
    radio_temp_df = radio_temp_df.append(table_df)
    
    

In [5]:
# Save dataframe to final so don't have to rescrape
radio_df = radio_temp_df.copy()

In [6]:
#Rename columns and display table
radio_df = radio_df.rename(columns={radio_df.columns[0]:"Call_sign",radio_df.columns[1]:"Frequency", radio_df.columns[2]:"Location",radio_df.columns[3]:"License",radio_df.columns[4]:"Format",radio_df.columns[5]:"State"})
# Delete rows with Call sign
radio_df = radio_df[radio_df.Call_sign !='Call sign']
radio_df

,Call_sign,Frequency,Location,License,Format,State
1,KRLE,89.7 FM,Carbon Hill,Educational Media Foundation,Contemporary Christian (K-LOVE),Alabama
2,WAAO-FM,93.7 FM,Andalusia,"Three Notch Communications, LLC",Country,Alabama
3,WAAX,570 AM,Gadsden,Capstar TX LLC,News/Talk,Alabama
4,WABD,97.5 FM,Mobile,Cumulus Licensing LLC,Top 40 (CHR),Alabama
5,WABF,1480 AM,Mobile,Eternity Record Company LLC,Standards,Alabama
6,WACQ,580 AM,Tuskegee,"Tiger Communications, Inc.",Classic Hits,Alabama
7,WACT,1420 AM,Tuscaloosa,Capstar TX LLC,Adult Contemporary,Alabama
8,WACV,93.1 FM,Coosada,"Liberty Acquisitions 825, LLC",News/Talk,Alabama
9,WAFN-FM,92.7 FM,Arab,"Fun Media Group, Inc.",Oldies,Alabama
10,WAGF-FM,101.3 FM,Dothan,"Wilson Broadcasting Co., Inc.","R&B, Motown oldies, blues, gospel music, and s...",Alabama


In [7]:
# add revised format and yes or no column for target audience

#import radio csv as a dataframe
formats_df = pd.read_csv('formats.csv',encoding='latin_1')

#join radio dataframe on "Format"
radio_format_df = radio_df.merge(formats_df, on="Format", how="left")

#drop "Format"
radio_format_df = radio_format_df.drop(['Format'], axis=1)

#change Revised Format to Format
radio_format_df = radio_format_df.rename(columns={"Revised_Format":"Format"})

# Fill in values for NaN
radio_format_df.Format.fillna(value="Other", inplace=True)
radio_format_df.Target_format.fillna(value="No", inplace=True)

radio_format_df


,Call_sign,Frequency,Location,License,State,Format,Target_format
0,KRLE,89.7 FM,Carbon Hill,Educational Media Foundation,Alabama,Religious,No
1,WAAO-FM,93.7 FM,Andalusia,"Three Notch Communications, LLC",Alabama,Country,Yes
2,WAAX,570 AM,Gadsden,Capstar TX LLC,Alabama,Talk,No
3,WABD,97.5 FM,Mobile,Cumulus Licensing LLC,Alabama,Top40,No
4,WABF,1480 AM,Mobile,Eternity Record Company LLC,Alabama,Other,No
5,WACQ,580 AM,Tuskegee,"Tiger Communications, Inc.",Alabama,Other,No
6,WACT,1420 AM,Tuscaloosa,Capstar TX LLC,Alabama,Other,No
7,WACV,93.1 FM,Coosada,"Liberty Acquisitions 825, LLC",Alabama,Talk,No
8,WAFN-FM,92.7 FM,Arab,"Fun Media Group, Inc.",Alabama,Oldies,No
9,WAGF-FM,101.3 FM,Dothan,"Wilson Broadcasting Co., Inc.",Alabama,Variety,No


In [8]:
# Counts of target stations - Blues, bluegrass, country, cajun & sports

# Keeping only yes target
target_yes_df = radio_format_df.loc[radio_format_df["Target_format"] == "Yes", :]
# Grouping by state
target_yes_count_df = target_yes_df.groupby(['State'])
# Count by state
target_yes_for_json = target_yes_count_df.count()
# Code to make a stable dataframe
target_yes_for_json.reset_index(inplace=True)
target_yes_for_json

,State,Call_sign,Frequency,Location,License,Format,Target_format
0,Alabama,69,69,69,69,69,69
1,Alaska,17,17,17,17,17,17
2,Arizona,44,44,44,44,44,44
3,Arkansas,85,85,85,85,85,85
4,California,88,88,88,88,88,88
5,Colorado,58,58,58,58,58,58
6,Connecticut,5,5,5,5,5,5
7,Delaware,4,4,4,4,4,4
8,Florida,103,103,103,103,103,103
9,Georgia,93,93,93,93,93,93


In [9]:
# Create JSON for target count
# Select 2 columns
target_yes_for_json_2 = target_yes_for_json[target_yes_for_json.columns[0:2]]
target_yes_for_json_2 = target_yes_for_json_2.rename(columns={target_yes_for_json.columns[1]:"Target"})
target_yes_for_json_2

,State,Target
0,Alabama,69
1,Alaska,17
2,Arizona,44
3,Arkansas,85
4,California,88
5,Colorado,58
6,Connecticut,5
7,Delaware,4
8,Florida,103
9,Georgia,93


In [10]:
# Create json file for target counts and download
target_yes_for_json_file = target_yes_for_json_2.to_json(orient='records',path_or_buf='Target_Counts.json')

In [11]:
# Counts of each format by state

# Grouping by state and station
all_formats_df = radio_format_df.groupby(['State','Format'])
# Count by state
all_formats_for_json = all_formats_df.count()
# Code to make a stable dataframe
all_formats_for_json.reset_index(inplace=True)
all_formats_for_json

,State,Format,Call_sign,Frequency,Location,License,Target_format
0,Alabama,Alternative,1,1,1,1,1
1,Alabama,Classical,2,2,2,2,2
2,Alabama,Contemporary,10,10,10,10,10
3,Alabama,Country,46,46,46,46,46
4,Alabama,Jazz,4,4,4,4,4
5,Alabama,Oldies,17,17,17,17,17
6,Alabama,Other,90,90,90,90,90
7,Alabama,R&B_Soul,2,2,2,2,2
8,Alabama,Religious,118,118,118,118,118
9,Alabama,Rock,5,5,5,5,5


In [12]:
# Create JSON for format counts

# Select 3 columns
all_formats_for_json_2 = all_formats_for_json[all_formats_for_json.columns[0:3]]
all_formats_for_json_2 = all_formats_for_json_2.rename(columns={all_formats_for_json_2.columns[2]:"Format_Count"})
all_formats_for_json_2

,State,Format,Format_Count
0,Alabama,Alternative,1
1,Alabama,Classical,2
2,Alabama,Contemporary,10
3,Alabama,Country,46
4,Alabama,Jazz,4
5,Alabama,Oldies,17
6,Alabama,Other,90
7,Alabama,R&B_Soul,2
8,Alabama,Religious,118
9,Alabama,Rock,5


In [13]:
# Create json file for target counts and download
all_formats_for_json_file = all_formats_for_json_2.to_json(orient='records',path_or_buf='Station_Counts.json')

## Radio data to sql

In [14]:
# Import Dependencies
import sqlite3 as lite
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String, Float, DATETIME, NUMERIC
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import *
import datetime

# Create engine and automap_base
engine = create_engine('sqlite:///Radio.db', echo=True)
conn = engine.connect()

# Create Table

Base = declarative_base()

2019-04-18 18:49:19,455 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-18 18:49:19,455 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 18:49:19,456 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-18 18:49:19,457 INFO sqlalchemy.engine.base.Engine ()


In [15]:
radio_format_df.to_sql('tbl_Radio', con=engine)

2019-04-18 18:49:22,431 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("tbl_Radio")
2019-04-18 18:49:22,432 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 18:49:22,437 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "tbl_Radio" (
	"index" BIGINT, 
	"Call_sign" TEXT, 
	"Frequency" TEXT, 
	"Location" TEXT, 
	"License" TEXT, 
	"State" TEXT, 
	"Format" TEXT, 
	"Target_format" TEXT
)


2019-04-18 18:49:22,438 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 18:49:22,442 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04-18 18:49:22,443 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_tbl_Radio_index" ON "tbl_Radio" ("index")
2019-04-18 18:49:22,444 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 18:49:22,448 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04-18 18:49:22,463 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-18 18:49:22,597 INFO sqlalchemy.engine.base.Engine INSERT INTO "tbl_Radio" ("index", "Call_sign", "Frequency", "Location", "License", "State", "Forma

In [16]:
# double check the new table in db
# import dependencies
import sqlite3
import sys

# Create the connections
conn = sqlite3.connect('Radio.db')
curs = conn.cursor()

# Print out data from table BME_DATA

for row in curs.execute("SELECT * FROM tbl_Radio"):
   print (row)

conn.close()

(0, 'KRLE', '89.7 FM', 'Carbon Hill', 'Educational Media Foundation', 'Alabama', 'Religious', 'No')
(1, 'WAAO-FM', '93.7 FM', 'Andalusia', 'Three Notch Communications, LLC', 'Alabama', 'Country', 'Yes')
(2, 'WAAX', '570 AM', 'Gadsden', 'Capstar TX LLC', 'Alabama', 'Talk', 'No')
(3, 'WABD', '97.5 FM', 'Mobile', 'Cumulus Licensing LLC', 'Alabama', 'Top40', 'No')
(4, 'WABF', '1480 AM', 'Mobile', 'Eternity Record Company LLC', 'Alabama', 'Other', 'No')
(5, 'WACQ', '580 AM', 'Tuskegee', 'Tiger Communications, Inc.', 'Alabama', 'Other', 'No')
(6, 'WACT', '1420 AM', 'Tuscaloosa', 'Capstar TX LLC', 'Alabama', 'Other', 'No')
(7, 'WACV', '93.1 FM', 'Coosada', 'Liberty Acquisitions 825, LLC', 'Alabama', 'Talk', 'No')
(8, 'WAFN-FM', '92.7 FM', 'Arab', 'Fun Media Group, Inc.', 'Alabama', 'Oldies', 'No')
(9, 'WAGF-FM', '101.3 FM', 'Dothan', 'Wilson Broadcasting Co., Inc.', 'Alabama', 'Variety', 'No')
(10, 'WAGG', '610 AM', 'Birmingham', 'SM-WAGG, LLC', 'Alabama', 'Religious', 'No')
(11, 'WAGH', '101

(1779, 'KMAK', '100.3 FM', 'Orange Cove', 'KMAK-FM, LLC', 'California', 'Spanish', 'No')
(1780, 'KMBX', '700 AM', 'Soledad', 'Entravision Holdings, LLC', 'California', 'Religious', 'No')
(1781, 'KMDR', '95.1 FM', 'McKinleyville', 'Mad River Radio, Inc.', 'California', 'Contemporary', 'No')
(1782, 'KMEC-LP', '105.1 FM', 'Ukiah', 'Mendocino Environmental Center', 'California', 'Variety', 'No')
(1783, 'KMEL', '106.1 FM', 'San Francisco', 'AMFM Broadcasting Licenses, LLC', 'California', 'Urban', 'No')
(1784, 'KMEN', '100.5 FM', 'Mendota', 'Casa Media Partners, LLC', 'California', 'Spanish', 'No')
(1785, 'KMET', '1490 AM', 'Banning', 'Rocking M Media, LLC', 'California', 'Talk', 'No')
(1786, 'KMGV', '97.9 FM', 'Fresno', 'Cumulus Licensing LLC', 'California', 'Other', 'No')
(1787, 'KMIX', '100.9 FM', 'Tracy', 'Entravision Holdings, LLC', 'California', 'Spanish', 'No')
(1788, 'KMJ', '580 AM', 'Fresno', 'Cumulus Licensing LLC', 'California', 'Talk', 'No')
(1789, 'KMJ-FM', '105.9 FM', 'Fresno',

(2840, 'WUCS', '97.9 FM', 'Windsor Locks', 'Capstar TX LLC', 'Connecticut', 'Sports', 'Yes')
(2841, 'WVOF', '88.5 FM', 'Fairfield', 'Fairfield University', 'Connecticut', 'Variety', 'No')
(2842, 'WWBW-LP', '97.1 FM', 'Higganum', 'Connecticut River Educational Radio, Inc.', 'Connecticut', 'Religious', 'No')
(2843, 'WWCO', '1240 AM', 'Waterbury', 'RedWolf Radio, LLC', 'Connecticut', 'Spanish', 'No')
(2844, 'WWEB', '89.9 FM', 'Wallingford', 'Choate Rosemary Hall Foundation', 'Connecticut', 'Variety', 'No')
(2845, 'WWMM-LP', '107.5 FM', 'Collinsville', 'Huckleberry Hill Music Society, Inc.', 'Connecticut', 'Variety', 'No')
(2846, 'WWPT', '90.3 FM', 'Westport', 'Westport Ct. Board of Education', 'Connecticut', 'Rock', 'No')
(2847, 'WWUH', '91.3 FM', 'West Hartford', 'University of Hartford', 'Connecticut', 'Jazz', 'No')
(2848, 'WWYZ', '92.5 FM', 'Waterbury', 'Capstar TX LLC', 'Connecticut', 'Country', 'Yes')
(2849, 'WXCI', '91.7 FM', 'Danbury', 'Western Connecticut State University', 'Conne

(4373, 'KDDE-LP', '96.7 FM', 'American Falls', 'Templo Emanuel Asambleas de Dios', 'Idaho', 'Spanish', 'No')
(4374, 'KDIL', '940 AM', 'Jerome', 'Maria Rosario Ortega', 'Idaho', 'Other', 'No')
(4375, 'KDKI-LP', '103.9 FM', 'Twin Falls', 'Tamarack Community Broadcasting, Inc.', 'Idaho', 'Jazz', 'No')
(4376, 'KDPI', '88.5 FM', 'Ketchum', 'KDPI Drop-In Radio, Inc.', 'Idaho', 'Other', 'No')
(4377, 'KDZY', '98.3 FM', 'McCall', 'Inspirational Family Radio, Inc.', 'Idaho', 'Country', 'Yes')
(4378, 'KECH-FM', '95.3 FM', 'Sun Valley', 'RP Broadcasting LS, LLC', 'Idaho', 'Rock', 'No')
(4379, 'KEDJ', '103.1 FM', 'Jerome', 'KART Broadcasting Co., Inc.', 'Idaho', 'Rock', 'No')
(4380, 'KEFX', '88.9 FM', 'Twin Falls', 'Calvary Chapel of Twin Falls, Inc.', 'Idaho', 'Religious', 'No')
(4381, 'KEII', '690 AM', 'Blackfoot', 'Riverbend Communications, LLC', 'Idaho', 'Other', 'No')
(4382, 'KEZJ', '1450 AM', 'Twin Falls', 'Salt & Light Radio, Inc.', 'Idaho', 'Religious', 'No')
(4383, 'KEZJ-FM', '95.7 FM', 'T

(6306, 'WLFX', '106.7 FM', 'Berea', 'Wallingford Communications, LLC', 'Kentucky', 'Top40', 'No')
(6307, 'WLGC-FM', '105.7 FM', 'Greenup', 'Greenup County Broadcasting, Inc.', 'Kentucky', 'Top40', 'No')
(6308, 'WLGX', '100.5 FM', 'Louisville', 'Clear Channel Broadcasting Licenses, Inc.', 'Kentucky', 'Contemporary', 'No')
(6309, 'WLHN-LP', '95.3 FM', 'Brandenburg', 'Meade County Catholic Radio, Inc.', 'Kentucky', 'Religious', 'No')
(6310, 'WLJC', '102.1 FM', 'Beattyville', 'Hour of Harvest, Incorporated', 'Kentucky', 'Religious', 'No')
(6311, 'WLKS-FM', '102.9 FM', 'West Liberty', 'Morgan County Industries, Inc.', 'Kentucky', 'Country', 'Yes')
(6312, 'WLKT', '104.5 FM', 'Lexington-Fayette', 'Citicasters Licenses, Inc.', 'Kentucky', 'Top40', 'No')
(6313, 'WLLE', '102.1 FM', 'Mayfield', 'Bristol Broadcasting Company, Inc.', 'Kentucky', 'Country', 'Yes')
(6314, 'WLLK-FM', '102.3 FM', 'Somerset', 'Capstar TX LLC', 'Kentucky', 'Contemporary', 'No')
(6315, 'WLLV', '1240 AM', 'Louisville', 'An

...

(9338, 'KDNE', '91.9 FM', 'Crete', 'Doane College Board of Trustees', 'Nebraska', 'Rock', 'No')
(9339, 'KELN', '97.1 FM', 'North Platte', 'Eagle Communications', 'Nebraska', 'Contemporary', 'No')
(9340, 'KETT', '99.3 FM', 'Mitchell', 'Legacy Communications, LLC', 'Nebraska', 'Contemporary', 'No')
(9341, 'KEVH-LP', '100.7 FM', 'Hastings', 'Educational Voice of Hastings', 'Nebraska', 'Religious', 'No')
(9342, 'KEXL', '97.5 FM', 'Pierce', 'WJAG Incorporated', 'Nebraska', 'Contemporary', 'No')
(9343, 'KEZO-FM', '92.3 FM', 'Omaha', 'Scripps Broadcasting Holdings LLC', 'Nebraska', 'Rock', 'No')
(9344, 'KFAB', '1110 AM', 'Omaha', 'Capstar TX LLC', 'Nebraska', 'Talk', 'No')
(9345, 'KFFF', '93.3 FM', 'Bennington', 'Capstar TX LLC', 'Nebraska', 'Country', 'Yes')
(9346, 'KFGE', '98.1 FM', 'Milford', 'NRG License Sub, LLC', 'Nebraska', 'Country', 'Yes')
(9347, 'KFHC', '88.1 FM', 'Ponca', 'St. Gabriel Communications Ltd.', 'Nebraska', 'Other', 'No')
(9348, 'KFJS', '90.1 FM', 'North Platte', 'VSS Ca

(11097, 'WLJF-LP', '100.7 FM', 'Greensboro', 'Love and Faith Christian Fellowship', 'North Carolina', 'Religious', 'No')
(11098, 'WLJZ-LP', '107.1 FM', 'Salisbury', 'Livingstone College', 'North Carolina', 'Variety', 'No')
(11099, 'WLKO', '102.9 FM', 'Hickory', 'Capstar TX LLC', 'North Carolina', 'Top40', 'No')
(11100, 'WLLN', '1370 AM', 'Lillington', 'Estuardo Valdemar Rodriguez', 'North Carolina', 'Spanish', 'No')
(11101, 'WLLQ', '1530 AM', 'Chapel Hill', 'Estuardo Valdemar Rodriguez and Leonor Rodriguez', 'North Carolina', 'Spanish', 'No')
(11102, 'WLLY', '1350 AM', 'Wilson', 'Estuardo Valdemar Rodriguez and Leonor Rodriguez, JT Tenants', 'North Carolina', 'Religious', 'No')
(11103, 'WLNC', '1300 AM', 'Laurinburg', 'Scotland Broadcasting Company, Inc.', 'North Carolina', 'Contemporary', 'No')
(11104, 'WLNK', '107.9 FM', 'Charlotte', 'Entercom License, LLC', 'North Carolina', 'Contemporary', 'No')
(11105, 'WLNR', '1230 AM', 'Kinston', 'Estuardo Valdemar Rodriguez & Leonor Rodriguez',

(12735, 'KZLY', '99.5 FM', 'Ione', 'Xana Oregon, LLC', 'Oregon', 'Spanish', 'No')
(12736, 'KZRI', '88.7 FM', 'Sandy', 'Educational Media Foundation', 'Oregon', 'Religious', 'No')
(12737, 'KZTB', '97.9 FM', 'Milton-Freewater', 'Bustos Media Holdings, LLC', 'Oregon', 'Spanish', 'No')
(12738, 'KZZF-LP', '107.7 FM', 'Klamath Falls', 'Jesus Radio, Inc.', 'Oregon', 'Religious', 'No')
(12739, 'KZZR', '94.3 FM', 'Government Camp', 'Bustos Media Holdings, L.L.C.', 'Oregon', 'Spanish', 'No')
(12740, 'KDKA', '1020 AM', 'Pittsburgh', 'Entercom License, LLC', 'Pennsylvania', 'Talk', 'No')
(12741, 'KDKA-FM', '93.7 FM', 'Pittsburgh', 'Entercom License, LLC', 'Pennsylvania', 'Sports', 'Yes')
(12742, 'KQV', '1410 AM', 'Pittsburgh', 'Broadcast Educational Communications, Inc.', 'Pennsylvania', 'Other', 'No')
(12743, 'KYW', '1060 AM', 'Philadelphia', 'Entercom License, LLC', 'Pennsylvania', 'Talk', 'No')
(12744, 'WABT', '96.7 FM', 'Lehman Township', 'Neversink Radio, LLC', 'Pennsylvania', 'Top40', 'No')


(14336, 'KAND', '1340 AM', 'Corsicana', 'New Century Broadcasting, LLC', 'Texas', 'Country', 'Yes')
(14337, 'KANI', '1500 AM', 'Wharton', 'Martin Broadcasting, Inc.', 'Texas', 'Religious', 'No')
(14338, 'KANJ', '91.1 FM', 'Giddings', 'Houston Christian Broadcasters, Inc.', 'Texas', 'Other', 'No')
(14339, 'KAPN', '107.3 FM', 'Caldwell', 'Brazos Valley Communications, Ltd.', 'Texas', 'Other', 'No')
(14340, 'KAPW', '99.3 FM', 'White Oak', 'Reynolds Radio, Inc.', 'Texas', 'Spanish', 'No')
(14341, 'KAQD', '91.3 FM', 'Abilene', 'American Family Association', 'Texas', 'Religious', 'No')
(14342, 'KARX', '107.1 FM', 'Canyon', 'Cumulus Licensing LLC', 'Texas', 'Country', 'Yes')
(14343, 'KASE-FM', '100.7 FM', 'Austin', 'Capstar TX LLC', 'Texas', 'Country', 'Yes')
(14344, 'KATG', '88.1 FM', 'Elkhart', 'American Family Association', 'Texas', 'Religious', 'No')
(14345, 'KATH', '910 AM', 'Frisco', 'Chatham Hill Foundation, Inc.', 'Texas', 'Religious', 'No')
(14346, 'KATP', '101.9 FM', 'Amarillo', 'To

(16167, 'WVES', '101.5 FM', 'Chincoteague', 'Jackman Holding Company, LLC', 'Virginia', 'Top40', 'No')
(16168, 'WVGM', '1320 AM', 'Lynchburg', '3 Daughters Media, Inc.', 'Virginia', 'Sports', 'Yes')
(16169, 'WVHK', '100.7 FM', 'Christiansburg', 'Cumulus Licensing LLC', 'Virginia', 'Other', 'No')
(16170, 'WVHL', '92.9 FM', 'Farmville', 'The Farmville Herald, Inc.', 'Virginia', 'Country', 'Yes')
(16171, 'WVHT', '100.5 FM', 'Norfolk', 'MHR License LLC', 'Virginia', 'Other', 'No')
(16172, 'WVKL', '95.7 FM', 'Norfolk', 'Entercom License, LLC', 'Virginia', 'Other', 'No')
(16173, 'WVLS', '89.7 FM', 'Monterey', 'Pocahontas Communications Cooperative Corp.', 'Virginia', 'Other', 'No')
(16174, 'WVMP', '101.5 FM', 'Vinton', 'Community Media Group, LLC', 'Virginia', 'Oldies', 'No')
(16175, 'WVNZ', '1320 AM', 'Richmond', 'TBLC Virginia Holdings, LLC', 'Virginia', 'Spanish', 'No')
(16176, 'WVRI', '90.9 FM', 'Clifton Forge', 'Liberty University, Inc.', 'Virginia', 'Religious', 'No')
(16177, 'WVRS', '